In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, RobustScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn import datasets
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, confusion_matrix, recall_score, precision_score, f1_score
import missingno as msno
import pgeocode
import xgboost as xgb
from sklearn.preprocessing import TargetEncoder, LabelEncoder
import scipy.stats as stats
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
import re
import lightgbm as lgb


In [3]:
data = pd.read_csv('datahotencoded.csv')

In [4]:
data = data.drop(["LoanNr_ChkDgt", "domaine d’activite","FranchiseCode", "ChgOffDate", "DisbursementDate", "DisbursementGross", "BalanceGross", "ChgOffPrinGr", "SBA_Appv", "Name", "City", "Zip", "Bank", "BankState", "NAICS", "RevLineCr", "ApprovalDate"], axis=1)

In [5]:
data.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 899164 entries, 0 to 899163
Data columns (total 83 columns):
 #   Column                                                                               Non-Null Count   Dtype  
---  ------                                                                               --------------   -----  
 0   ApprovalFY                                                                           899164 non-null  int64  
 1   Term                                                                                 899164 non-null  int64  
 2   NoEmp                                                                                899164 non-null  int64  
 3   NewExist                                                                             899028 non-null  float64
 4   CreateJob                                                                            899164 non-null  int64  
 5   RetainedJob                                                                    

In [6]:
data.columns = data.columns.str.replace(' ', '_')
data.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 899164 entries, 0 to 899163
Data columns (total 83 columns):
 #   Column                                                                               Non-Null Count   Dtype  
---  ------                                                                               --------------   -----  
 0   ApprovalFY                                                                           899164 non-null  int64  
 1   Term                                                                                 899164 non-null  int64  
 2   NoEmp                                                                                899164 non-null  int64  
 3   NewExist                                                                             899028 non-null  float64
 4   CreateJob                                                                            899164 non-null  int64  
 5   RetainedJob                                                                    

In [7]:
X = data.drop('MIS_Status', axis=1)
y = data['MIS_Status']


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
# Créer des datasets LightGBM
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

# Définir les paramètres du modèle
params = {
    'objective': 'binary',  # Pour la classification binaire
    'metric': 'binary_logloss',
    'boosting_type': 'gbdt',
    'learning_rate': 0.1,
    'num_leaves': 31,
    'verbose': -1
}


In [10]:
# Entraîner le modèle
model = lgb.LGBMClassifier(
    objective='binary',
    metric='binary_logloss',
    boosting_type='gbdt',
    learning_rate=0.1,
    num_leaves=31,
    n_estimators=100,  # Correspond à num_boost_round
    early_stopping_rounds=10,  # Arrêt précoce
    verbose=10
)



In [12]:
model.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    eval_metric='binary_logloss',
    
)

[LightGBM] [Warning] early_stopping_round is set=10, early_stopping_rounds=10 will be ignored. Current value: early_stopping_round=10


[LightGBM] [Fatal] Do not support special JSON characters in feature name.


LightGBMError: Do not support special JSON characters in feature name.